In [ ]:
import http.server
from http.server import HTTPServer, SimpleHTTPRequestHandler
import ssl
import base64

key = "admin:admin"
encodedKey = "Basic " + str(base64.b64encode(key.encode("utf-8")),"utf-8")

class RequestHandler(http.server.SimpleHTTPRequestHandler):
    def do_HEAD(self):
        print ("Send Header")
        self.send_response(200)
        self.send_header('Content-type', 'text/html')
        self.end_headers()

    def do_AUTHHEAD(self):
        print ("Test")
        self.send_response(401)
        self.send_header('WWW-Authenticate', 'Basic realm=\"Test\"')
        self.send_header('Content-type', 'text/html')
        self.end_headers()

    def do_GET(self):
        if self.headers.get('Authorization') == None:
            self.do_AUTHHEAD()
            self.wfile.write(b'No Auth Header Received')
            pass
        elif self.headers.get('Authorization') == encodedKey:
            return SimpleHTTPRequestHandler.do_GET(self)
            pass
        else:
            self.do_AUTHHEAD()
            print(self.headers.get('Authorization'))
            self.wfile.write(b'Not Authenticated')
            pass

port = 1991
httpd = http.server.HTTPServer(('', port), RequestHandler)
print("Serving at port", port)
httpd.socket = ssl.wrap_socket(httpd.socket, keyfile = "key.pem", certfile = "certificate.pem", server_side=True)
httpd.serve_forever()


Serving at port 1991
Test


127.0.0.1 - - [24/Sep/2019 07:25:37] "GET / HTTP/1.1" 401 -
127.0.0.1 - - [24/Sep/2019 07:25:44] "GET / HTTP/1.1" 200 -
127.0.0.1 - - [24/Sep/2019 07:25:45] code 404, message File not found
127.0.0.1 - - [24/Sep/2019 07:25:45] "GET /favicon.ico HTTP/1.1" 404 -
